# GPT functions

With just a dialog of text messages you can do a lot, but it's difficult to extend the capabilities of GPT. FOr this purpose GPT has a mechanism called functions, they're similar to, but not the same as plugins. Functions allow you to include your own code into the answers of GPT. The mechanism is a bit indirect though.

1. First you explain what functions you have to offer, describing in plain text the purpose, its inputs and outputs.
2. GPT may reply with a request to apply a function, together with its inputs. This is instead of a normal reply.
3. You can run your own python code and reply with the function output
4. GPT includes the reply into an answer to your question.

In this notebook we follow the example of https://dekraker.medium.com/fun-with-functions-almost-everything-about-gpt-api-functions-619216eaebe where we extend GPT with the capability to forecast the weather. 

More info can be found at:
- https://platform.openai.com/docs/quickstart
- https://platform.openai.com/docs/api-reference

In [1]:
# installation on colab
# run this cell to install the required packages, but only once
# You may have to restart the runtime after running this cell (Runtime -> Restart runtime)
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
  %pip install openai
  %pip install requests
  # getpass already installed
else:
  print('Make sure that you have installed the required packages listed in requirements.txt')

Make sure that you have installed the required packages listed in requirements.txt


In [16]:
#imports
import openai
import getpass
from IPython.display import display, Markdown
import requests

In [3]:
#input your API key
openai.api_key = getpass.getpass("Enter your API key: ")

In [4]:
# This code was copied from: https://dekraker.medium.com/fun-with-functions-almost-everything-about-gpt-api-functions-619216eaebe
# GPT will read this to understand what Functions we define.
functions = [{
    "name": "get_current_weather",
    "description": "Get the current weather",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The four-digit ICAO Airport code for a METAR weather station. \
                Example: KSEA. If Airport code is not provided, infer it from other available location data \
                and your knowledge."
            },
        },
        "required": ["location"]
    }
}]

In [28]:
model = "gpt-3.5-turbo-0613" #TODO do we need this specific model?
max_tokens = 500 # More tokens are needed to include the function definition

messages = []
messages.append({"role": "system", "content": "You are a helpful assistant."})
messages.append({"role": "user", "content": "Does it rain in Delft?"}) 

In [38]:
# Now ask GPT to generate a response. We expect it to reply with a function call
response = openai.ChatCompletion.create( 
    model=model,
    messages=messages,
    max_tokens=max_tokens,
    functions=functions
)

print("Full response:",response)
print("=====================================")
finish_reason = response["choices"][0]["finish_reason"]
print("What was the reason to stop:",finish_reason)
function_name = "" # we will store the function name here
function_args = "" # we will store the location here
location = "" # we will store the location here
if finish_reason == "function_call":
  print("We received a function call!")
  reply = response["choices"][0]["message"] # look for info in the reply structure
  messages.append(reply) # add the reply to the messages
  function_call = reply["function_call"]
  function_name = function_call["name"]
  function_args = function_call["arguments"]
  print("The function name is:",function_name)
  location=eval(function_args)["location"]
  print("The location is (as an aviation code):",location)
  print("The function arguments are:",function_args)
elif finish_reason == "max_tokens":
  print("We reached the maximum number of tokens before a full answer or function_call.")
elif finish_reason == "stop":
  print("We received a full answer, but no function call.")
else:
  print("We received an unexpected response:",finish_reason)

print("Code EHRD is Rotterdam The Hague Airport, but it does not always work.")

Full response: {
  "id": "chatcmpl-7aV6gRPdBQA2B9iiJV7zavcUr6aGa",
  "object": "chat.completion",
  "created": 1688933738,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"EHRD\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 100,
    "completion_tokens": 18,
    "total_tokens": 118
  }
}
What was the reason to stop: function_call
We received a function call!
The function name is: get_current_weather
The location is (as an aviation code): EHRD
The function arguments are: {
  "location": "EHRD"
}
Code EHRD is Rotterdam The Hague Airport, but it does not always work.


In [41]:
# Now we use our function name and arguments to generate a response
function_response = ""

if function_name == "get_current_weather":
    # Do stuff for this function
    print("The function name is get_current_weather.")
    print("The location code is:",location)

    import requests
    import urllib.parse

    # Set up the URL, passing on the location variable such as KDTW
    url = f'https://www.aviationweather.gov/adds/dataserver_current/httpparam?dataSource=metars&requestType=retrieve&format=xml&hoursBeforeNow=3&mostRecent=true&stationString={location}'

    # Use requests to fetch the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
      function_response = response.text
      print("response is:",function_response)

    else:
      print("Error making the request.")
elif function_name=="":
  print("No function call received.")
else:
  print("We received an unexpected function name")

# function reply to GPT
messages.append({"role": "system", "name": function_name, "content": function_response})

The function name is get_current_weather.
The location code is: EHRD
response is: <?xml version="1.0" encoding="UTF-8"?>
<response xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="1.2" xsi:noNamespaceSchemaLocation="http://www.aviationweather.gov/static/adds/schema/metar1_2.xsd">
  <request_index>2197922</request_index>
  <data_source name="metars" />
  <request type="retrieve" />
  <errors />
  <warnings />
  <time_taken_ms>1</time_taken_ms>
  <data num_results="1">
    <METAR>
      <raw_text>EHRD 091955Z AUTO 29003KT 250V340 9999 NSC 18/15 Q1020 TEMPO 7000 -SHRA SCT030CB</raw_text>
      <station_id>EHRD</station_id>
      <observation_time>2023-07-09T19:55:00Z</observation_time>
      <latitude>51.95</latitude>
      <longitude>4.45</longitude>
      <temp_c>18.0</temp_c>
      <dewpoint_c>15.0</dewpoint_c>
      <wind_dir_degrees>290</wind_dir_degrees>
      <wind_speed_kt>3</wind_speed_kt>
      <visibility_statute_mi>6.2

In [40]:
# Now we extract the weather information from the response
function_response  #Let's forget about this for now

'<?xml version="1.0" encoding="UTF-8"?>\r\n<response xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="1.2" xsi:noNamespaceSchemaLocation="http://www.aviationweather.gov/static/adds/schema/metar1_2.xsd">\r\n  <request_index>2116325</request_index>\r\n  <data_source name="metars" />\r\n  <request type="retrieve" />\r\n  <errors />\r\n  <warnings />\r\n  <time_taken_ms>2</time_taken_ms>\r\n  <data num_results="1">\r\n    <METAR>\r\n      <raw_text>EHRD 091955Z AUTO 29003KT 250V340 9999 NSC 18/15 Q1020 TEMPO 7000 -SHRA SCT030CB</raw_text>\r\n      <station_id>EHRD</station_id>\r\n      <observation_time>2023-07-09T19:55:00Z</observation_time>\r\n      <latitude>51.95</latitude>\r\n      <longitude>4.45</longitude>\r\n      <temp_c>18.0</temp_c>\r\n      <dewpoint_c>15.0</dewpoint_c>\r\n      <wind_dir_degrees>290</wind_dir_degrees>\r\n      <wind_speed_kt>3</wind_speed_kt>\r\n      <visibility_statute_mi>6.21</visibility_statute_mi

In [49]:
# Now ask GPT to generate the answer
# Now ask GPT to generate a response. We expect it to reply with a function call
response = openai.ChatCompletion.create( 
    model=model,
    messages=messages,
    max_tokens=max_tokens,
    functions=functions
)

#print("Full response:",response) # uncomment this line to see the full response


__User__: Does it rain in Delft?

__GPT__: According to the latest weather report, it is not currently raining in Delft. The observation shows clear skies (CLR) with a visibility of 6.21 statute miles.

In [50]:
# Write the question and answer to the screen
display(Markdown("__User__: "+messages[1]["content"]))
display(Markdown("__GPT__: "+response["choices"][0]["message"]["content"]))

__User__: Does it rain in Delft?

__GPT__: According to the latest weather report, it is not currently raining in Delft. The observation shows clear skies (CLR) with a visibility of 6.21 statute miles.